In [2]:
import json

from tqdm import tqdm
import tweepy
from dotenv import load_dotenv
import os

load_dotenv()  # take environment variables from .env.

True

In [36]:
DATA_PATH = 'data'
YEARS = [2016, 2020]

In [14]:
for year in YEARS:
    with open(os.path.join(DATA_PATH, f'polls_{year}_labeled.json')) as f:
        polls = json.load(f)
    author_ids = list(map(lambda x: x['meta']['author_id'], polls))
    with open(os.path.join(DATA_PATH, f'polls_{year}_author_ids.json'), 'w+') as f:
        json.dump(author_ids, f)

In [32]:
auth = tweepy.OAuth2BearerHandler(
    bearer_token= os.environ['BEARER'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
for year in YEARS:
    with open(os.path.join(DATA_PATH, f'polls_{year}_author_ids.json')) as f:
        author_ids = json.load(f)
    with open(os.path.join(DATA_PATH, f'follower_lists_{year}.jsonl'), 'w+', encoding='utf8') as f:
        for author_id in tqdm(author_ids, desc='checking followers of poll authors'):
            follower_ids = []
            for page in tweepy.Cursor(api.get_follower_ids, user_id = author_id, count = 5000).pages():
                follower_ids.extend(page)
                f.write(json.dumps({author_id:follower_ids}) + '\n')

checking followers of poll authors:   6%|▋         | 7/111 [00:04<01:48,  1.04s/it]Rate limit reached. Sleeping for: 721


In [19]:
# client = tweepy.Client(bearer_token=os.environ['BEARER'],
#                        wait_on_rate_limit=True)
# author_id = author_ids[0]
# user_fields = ['id']
# response = client.get_users_followers(id=author_id, user_fields=user_fields, max_results=1000)
# one_user = response.data[0]
# one_user.data
# next_token = response.meta['next_token']